In [70]:
# 6クラスラベルの再生成（background, conj, iris_vis, iris_occ, pupil_vis, pupil_occ）
from pathlib import Path
import shutil
import cv2
import numpy as np
import pandas as pd
import glob

IMAGES_DIR = Path('Images/images')
LABEL_SEG_DIR = Path('Images/labels_seg')
LABEL_OBB_DIR = Path('Images/labels_obb')

# 既存sixclsファイルのみ削除（ディレクトリは再利用）
for p in glob.glob(str(LABEL_SEG_DIR / '*_sixcls.png')):
    try:
        Path(p).unlink()
    except Exception:
        pass

# 画像リスト（image_metadata.csv を前段で作っている前提）
df = pd.read_csv('image_metadata.csv')

count = 0
for _, row in df.iterrows():
    fname = row['filename']
    stem = Path(fname).stem

    # 必要マスクのロード（512へは保存側でサイズ統一済み想定、なければここでresize）
    lid = cv2.imread(str(LABEL_SEG_DIR / f"{stem}_mask_lid.png"), 0)
    iris = cv2.imread(str(LABEL_OBB_DIR / f"{stem}_mask_iris.png"), 0)
    pupil = cv2.imread(str(LABEL_OBB_DIR / f"{stem}_mask_pupil.png"), 0)
    if lid is None or iris is None or pupil is None:
        continue

    H, W = lid.shape[:2]
    # 2値化
    lid_b = (lid > 0)
    iris_b = (iris > 0)
    pupil_b = (pupil > 0)

    # 6クラスID割当（ユーザ定義に準拠）
    # 0: background = ~lid & ~iris & ~pupil
    # 1: conj       =  lid & ~iris & ~pupil
    # 2: iris_vis   =  lid &  iris & ~pupil
    # 3: iris_occ   = ~lid &  iris & ~pupil
    # 4: pupil_vis  =  lid &  iris &  pupil
    # 5: pupil_occ  = ~lid &  iris &  pupil
    target = np.zeros((H, W), dtype=np.uint8)
    target[lid_b & ~iris_b & ~pupil_b] = 1
    target[lid_b &  iris_b & ~pupil_b] = 2
    target[~lid_b &  iris_b & ~pupil_b] = 3
    target[lid_b &  iris_b &  pupil_b] = 4
    target[~lid_b &  iris_b &  pupil_b] = 5

    # 保存（既存ディレクトリに上書き保存）
    out_path = LABEL_SEG_DIR / f"{stem}_sixcls.png"
    cv2.imwrite(str(out_path), target)
    count += 1

print(f"6クラスラベルを {count} 件生成しました: {LABEL_SEG_DIR}")


6クラスラベルを 1992 件生成しました: Images\labels_seg


# データ前処理

このノートブックでは、CVAT XML形式のアノテーションをパースし、トレーニング用ラベル（512×512, 6クラス）を生成します。

**⚠️ 重要**: このノートブックは「すべて実行（Run All）」で上から順番に実行されることを前提に設計されています。セルを個別に実行する場合は、依存関係に注意してください。

## 処理の流れ（更新）
1. CVAT XMLファイルの読み込みとパース
2. 患者IDの抽出（ファイル名から）
3. 画像・ラベルの処理（512×512へのリサイズ）
4. ラベル生成（`mask_lid`、`mask_iris`、`mask_pupil`、および6クラス`*_sixcls.png`）
5. 既存`*_sixcls.png`の削除 → 再生成（クリーンリビルド）
6. GroupKFold分割（患者IDベース）

## 6クラス定義
- 0: background = lid外 ∩ iris外 ∩ pupil外
- 1: conj       = lid内 ∩ iris外 ∩ pupil外
- 2: iris_vis   = lid内 ∩ iris内 ∩ pupil外
- 3: iris_occ   = lid外 ∩ iris内 ∩ pupil外
- 4: pupil_vis  = lid内 ∩ iris内 ∩ pupil内
- 5: pupil_occ  = lid外 ∩ iris内 ∩ pupil内

## 出力先
- `Images/labels_seg/`
  - `{stem}_mask_lid.png`
  - `{stem}_sixcls.png`（上記6クラスIDを持つ単一PNG）
- `Images/labels_obb/`
  - `{stem}_mask_iris.png`
  - `{stem}_mask_pupil.png`

## メタデータ（プロジェクトルート）
- `fold_indices.json` - 5-fold GroupKFold分割情報
- `image_metadata.csv` - 画像メタデータ（image_id, filename, patient_id, original_size）
- `patient_list.json` - 患者IDリスト


## 必要なライブラリのインポート


In [71]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import os
from pathlib import Path
import json
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import re

print("ライブラリのインポート完了")


ライブラリのインポート完了


## 設定とパス


In [72]:
# 画像サイズを512に統一（上書き）
IMAGE_SIZE = 512
print(f"[Override] 画像サイズを {IMAGE_SIZE}x{IMAGE_SIZE} に設定しました")


[Override] 画像サイズを 512x512 に設定しました


In [73]:
# パス設定
IMAGES_DIR = Path("Images/images")
EYELID_XML_PATH = Path("Images/eyelid_caruncle_seg_0-2000.xml")
IRIS_PUPIL_XML_PATH = Path("Images/obb_iris_pupil_1-3000.xml")

# 出力ディレクトリ
LABEL_SEG_DIR = Path("Images/labels_seg")
LABEL_OBB_DIR = Path("Images/labels_obb")
LABEL_SEG_DIR.mkdir(exist_ok=True)
LABEL_OBB_DIR.mkdir(exist_ok=True)

# 設定
IMAGE_SIZE = 640
MAX_IMAGE_ID = 1999  # image id 0-1999のみ使用
GLOBAL_SEED = 42

print(f"画像ディレクトリ: {IMAGES_DIR}")
print(f"出力ディレクトリ: {LABEL_SEG_DIR}, {LABEL_OBB_DIR}")
print(f"画像サイズ: {IMAGE_SIZE}x{IMAGE_SIZE}")


画像ディレクトリ: Images\images
出力ディレクトリ: Images\labels_seg, Images\labels_obb
画像サイズ: 640x640


## CVAT XMLパース関数


In [74]:
def parse_cvat_xml(xml_path):
    """
    CVAT XMLファイルをパースして、imageごとのアノテーション情報を返す
    
    Returns:
        dict: {image_id: {filename, width, height, annotations}}
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    annotations_dict = {}
    
    for image in root.findall('image'):
        image_id = int(image.get('id'))
        filename = image.get('name')
        width = int(image.get('width'))
        height = int(image.get('height'))
        
        annotations = {
            'filename': filename,
            'width': width,
            'height': height,
            'polygons': [],
            'ellipses': []
        }
        
        # polygon（眼瞼、涙丘）を取得
        for polygon in image.findall('polygon'):
            label = polygon.get('label')
            points_str = polygon.get('points')
            if points_str:
                annotations['polygons'].append({
                    'label': label,
                    'points': points_str
                })
        
        # ellipse（虹彩、瞳孔）を取得
        for ellipse in image.findall('ellipse'):
            label = ellipse.get('label')
            cx = float(ellipse.get('cx'))
            cy = float(ellipse.get('cy'))
            rx = float(ellipse.get('rx'))
            ry = float(ellipse.get('ry'))
            
            annotations['ellipses'].append({
                'label': label,
                'cx': cx,
                'cy': cy,
                'rx': rx,
                'ry': ry
            })
        
        annotations_dict[image_id] = annotations
    
    return annotations_dict

print("CVAT XMLパース関数を定義しました")


CVAT XMLパース関数を定義しました


## 患者ID抽出関数


In [75]:
def extract_patient_id(filename):
    """
    ファイル名から患者IDを抽出
    
    Args:
        filename: 例: '1-20141126-38-091804_...jpg'
    
    Returns:
        int: 患者ID（例: 1）
    """
    # ファイル名のbasename先頭の整数を抽出
    basename = os.path.basename(filename)
    match = re.match(r'^(\d+)', basename)
    if match:
        return int(match.group(1))
    else:
        # デフォルト値を返す（アノテがない画像の可能性）
        return -1

# テスト
test_filename = "1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554_L.jpg"
patient_id = extract_patient_id(test_filename)
print(f"テスト: {test_filename} → patient_id={patient_id}")


テスト: 1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554_L.jpg → patient_id=1


## ラスタライズ関数


In [76]:
def parse_points(points_str):
    """
    CVATのpoints形式の文字列をパース
    例: "281.31,446.31;274.60,448.40;..."
    """
    if not points_str:
        return []
    points = []
    for pair in points_str.split(';'):
        x, y = map(float, pair.split(','))
        points.append((x, y))
    return np.array(points, dtype=np.float32)

def rasterize_polygon(points_str, original_size, target_size=640):
    """
    polygonをラスタライズ
    
    Args:
        points_str: "x1,y1;x2,y2;..."形式の文字列
        original_size: (width, height) 元画像サイズ
        target_size: 目標画像サイズ
    
    Returns:
        np.ndarray: (target_size, target_size)のバイナリマスク
    """
    points = parse_points(points_str)
    if len(points) == 0:
        return np.zeros((target_size, target_size), dtype=np.uint8)
    
    # 元画像座標を正規化
    orig_w, orig_h = original_size
    points_norm = points.copy()
    points_norm[:, 0] = (points_norm[:, 0] / orig_w) * target_size
    points_norm[:, 1] = (points_norm[:, 1] / orig_h) * target_size
    
    # OpenCVで描画
    mask = np.zeros((target_size, target_size), dtype=np.uint8)
    points_int = points_norm.astype(np.int32)
    cv2.fillPoly(mask, [points_int], 255)
    
    return mask

def rasterize_ellipse(cx, cy, rx, ry, original_size, target_size=640):
    """
    ellipseをラスタライズ
    
    Args:
        cx, cy: 中心座標
        rx, ry: x半径, y半径
        original_size: (width, height)
        target_size: 目標画像サイズ
    
    Returns:
        np.ndarray: (target_size, target_size)のバイナリマスク
    """
    orig_w, orig_h = original_size
    
    # 座標を正規化
    cx_norm = (cx / orig_w) * target_size
    cy_norm = (cy / orig_h) * target_size
    rx_norm = (rx / orig_w) * target_size
    ry_norm = (ry / orig_h) * target_size
    
    # PILで描画（楕円はOpenCVよりPILの方が綺麗）
    mask = Image.new('L', (target_size, target_size), 0)
    draw = ImageDraw.Draw(mask)
    
    # 楕円の境界を計算
    bbox = [
        cx_norm - rx_norm,
        cy_norm - ry_norm,
        cx_norm + rx_norm,
        cy_norm + ry_norm
    ]
    
    draw.ellipse(bbox, fill=255)
    
    return np.array(mask, dtype=np.uint8)

print("ラスタライズ関数を定義しました")


ラスタライズ関数を定義しました


## CVAT XML読み込み


In [77]:
# 眼瞼・涙丘のアノテーションを読み込み
print("眼瞼・涙丘XMLファイルを読み込み中...")
eyelid_annotations = parse_cvat_xml(EYELID_XML_PATH)
print(f"眼瞼・涙丘: {len(eyelid_annotations)}件のアノテーション")

# 虹彩・瞳孔のアノテーションを読み込み
print("虹彩・瞳孔XMLファイルを読み込み中...")
iris_pupil_annotations = parse_cvat_xml(IRIS_PUPIL_XML_PATH)
print(f"虹彩・瞳孔: {len(iris_pupil_annotations)}件のアノテーション")

# image id 0-1999にフィルタ
filtered_image_ids = set(range(MAX_IMAGE_ID + 1))
print(f"\nimage id 0-{MAX_IMAGE_ID}のみを使用します")


眼瞼・涙丘XMLファイルを読み込み中...
眼瞼・涙丘: 4459件のアノテーション
虹彩・瞳孔XMLファイルを読み込み中...
虹彩・瞳孔: 4467件のアノテーション

image id 0-1999のみを使用します


## データ処理とラベル生成

各画像について、以下を処理します：
1. 眼瞼・涙丘の統合マスク生成
2. 虹彩・瞳孔の楕円マスク生成
3. 5クラス教師データ生成


In [78]:
def process_single_image(image_id):
    """
    単一画像を処理してラベルを生成
    
    Returns:
        dict or None: 処理結果、失敗時はNone
    """
    # eyelidアノテーションを取得
    if image_id not in eyelid_annotations:
        return None
    
    eyelid_ann = eyelid_annotations[image_id]
    filename = eyelid_ann['filename']
    original_size = (eyelid_ann['width'], eyelid_ann['height'])
    
    # 患者ID抽出
    patient_id = extract_patient_id(filename)
    
    # マスク生成
    mask_eyelid = np.zeros((IMAGE_SIZE, IMAGE_SIZE), dtype=np.uint8)
    mask_caruncle = np.zeros((IMAGE_SIZE, IMAGE_SIZE), dtype=np.uint8)
    
    for polygon in eyelid_ann['polygons']:
        label = polygon['label']
        points_str = polygon['points']
        mask = rasterize_polygon(points_str, original_size, IMAGE_SIZE)
        
        if label == 'Eyelid':
            mask_eyelid = np.maximum(mask_eyelid, mask)
        elif label == 'Caruncle':
            mask_caruncle = np.maximum(mask_caruncle, mask)
    
    # 眼瞼統合
    mask_lid = np.clip(mask_eyelid + mask_caruncle, 0, 255)
    
    # 虹彩・瞳孔処理
    iris_pupil_ann = iris_pupil_annotations.get(image_id)
    
    mask_iris = np.zeros((IMAGE_SIZE, IMAGE_SIZE), dtype=np.uint8)
    mask_pupil = np.zeros((IMAGE_SIZE, IMAGE_SIZE), dtype=np.uint8)
    
    if iris_pupil_ann:
        for ellipse in iris_pupil_ann['ellipses']:
            label = ellipse['label']
            mask = rasterize_ellipse(
                ellipse['cx'], ellipse['cy'],
                ellipse['rx'], ellipse['ry'],
                original_size, IMAGE_SIZE
            )
            
            if label == 'Iris':
                mask_iris = np.maximum(mask_iris, mask)
            elif label == 'Pupil':
                mask_pupil = np.maximum(mask_pupil, mask)
    
    # 一貫性保証: E_pupil ⊂ E_iris
    mask_pupil = np.bitwise_and(mask_pupil, mask_iris)
    
    # 5クラス教師データ生成
    # 注意：眼瞼マスクは「眼瞼縁に囲まれた部分（眼球の露出部分）」を表す
    # つまり、眼瞼マスクの中 = 見えている部分、眼瞼マスクの外 = 隠れている部分
    iris_vis = np.bitwise_and(mask_iris, mask_lid)         # 虹彩 ∩ 眼瞼 = 見える
    iris_occ = np.bitwise_and(mask_iris, 255 - mask_lid)   # 虹彩 ∩ (NOT 眼瞼) = 隠れている
    pupil_vis = np.bitwise_and(mask_pupil, mask_lid)       # 瞳孔 ∩ 眼瞼 = 見える
    pupil_occ = np.bitwise_and(mask_pupil, 255 - mask_lid) # 瞳孔 ∩ (NOT 眼瞼) = 隠れている
    
    return {
        'image_id': image_id,
        'filename': filename,
        'patient_id': patient_id,
        'mask_lid': mask_lid,
        
        'mask_iris': mask_iris,
        'mask_pupil': mask_pupil,
        'iris_vis': iris_vis,
        'iris_occ': iris_occ,
        'pupil_vis': pupil_vis,
        'pupil_occ': pupil_occ,
        'original_size': original_size
    }

print("データ処理関数を定義しました")


データ処理関数を定義しました


## 全データ処理とGroupKFold分割


In [79]:
# 全データを処理
results = []
for image_id in range(MAX_IMAGE_ID + 1):
    if image_id not in filtered_image_ids:
        continue
    
    result = process_single_image(image_id)
    if result is not None:
        results.append(result)
    
    if (image_id + 1) % 500 == 0:
        print(f"処理中: {image_id + 1}/{MAX_IMAGE_ID + 1}")

print(f"\n処理完了: {len(results)}件のデータ")

# DataFrame作成
df = pd.DataFrame(results)
print(f"\nデータフレーム作成完了")
print(f"患者数: {df['patient_id'].nunique()}")
print(f"\n最初の5件:")
print(df[['image_id', 'filename', 'patient_id']].head())

# 各患者の画像数を確認
patient_counts = df.groupby('patient_id').size()
print(f"\n患者ごとの画像数（上位10件）:")
print(patient_counts.sort_values(ascending=False).head(10))


処理中: 500/2000
処理中: 1000/2000
処理中: 1500/2000
処理中: 2000/2000

処理完了: 1992件のデータ

データフレーム作成完了
患者数: 122

最初の5件:
   image_id                                           filename  patient_id
0         0  1-20141126-38-091804_eb568e2ac952f8be45ec0ac9a...           1
1         1  1-20141126-38-091804_eb568e2ac952f8be45ec0ac9a...           1
2         2  1-20150121-38-142903_6e60b2355e174936406b708cf...           1
3         3  1-20150121-38-142903_6e60b2355e174936406b708cf...           1
4         4  1-20150121-38-142903_f27c8cf98eef934c557ffb70f...           1

患者ごとの画像数（上位10件）:
patient_id
208    102
164    100
136     74
156     71
195     70
151     64
145     58
160     50
18      46
162     43
dtype: int64


## GroupKFold分割（患者IDベース）


In [80]:
# GroupKFoldで分割
gkf = GroupKFold(n_splits=5)
groups = df['patient_id'].values
X = df.index.values
y = df['image_id'].values  # 実際のラベルは使わないが形式として

fold_indices = {}
for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    fold_indices[fold_idx] = {
        'train': train_idx.tolist(),
        'val': val_idx.tolist()
    }
    
    # 患者IDが重複していないか確認
    train_patients = set(df.iloc[train_idx]['patient_id'])
    val_patients = set(df.iloc[val_idx]['patient_id'])
    overlap = train_patients & val_patients
    
    print(f"Fold {fold_idx}:")
    print(f"  Train: {len(train_idx)}件（{len(train_patients)}患者）")
    print(f"  Val: {len(val_idx)}件（{len(val_patients)}患者）")
    print(f"  患者重複: {len(overlap)}件")
    print()

# fold_indicesを保存
with open('fold_indices.json', 'w') as f:
    json.dump(fold_indices, f, indent=2)

print("fold_indices.json に保存しました")


Fold 0:
  Train: 1593件（97患者）
  Val: 399件（25患者）
  患者重複: 0件

Fold 1:
  Train: 1594件（98患者）
  Val: 398件（24患者）
  患者重複: 0件

Fold 2:
  Train: 1594件（98患者）
  Val: 398件（24患者）
  患者重複: 0件

Fold 3:
  Train: 1593件（97患者）
  Val: 399件（25患者）
  患者重複: 0件

Fold 4:
  Train: 1594件（98患者）
  Val: 398件（24患者）
  患者重複: 0件

fold_indices.json に保存しました


## 既存ラベルの削除（再生成のため）


In [81]:
# 既存のラベルファイルを削除（再生成のため）
import shutil

if LABEL_SEG_DIR.exists():
    shutil.rmtree(LABEL_SEG_DIR)
    print(f"削除: {LABEL_SEG_DIR}")
LABEL_SEG_DIR.mkdir(exist_ok=True)

if LABEL_OBB_DIR.exists():
    shutil.rmtree(LABEL_OBB_DIR)
    print(f"削除: {LABEL_OBB_DIR}")
LABEL_OBB_DIR.mkdir(exist_ok=True)

print("既存のラベルディレクトリを削除しました")


削除: Images\labels_seg
削除: Images\labels_obb
既存のラベルディレクトリを削除しました


## ラベル保存


In [82]:
# ラベルを保存
def save_label(result, label_name, output_dir):
    """ラベル画像を保存"""
    mask = result[label_name]
    filename_base = os.path.splitext(result['filename'])[0]
    output_path = output_dir / f"{filename_base}_{label_name}.png"
    cv2.imwrite(str(output_path), mask)
    return output_path

# 各ラベルを保存
print("ラベル画像を保存中...")
for idx, result in enumerate(results):
    if (idx + 1) % 500 == 0:
        print(f"保存中: {idx + 1}/{len(results)}")
    
    # 眼瞼系ラベル
    save_label(result, 'mask_lid', LABEL_SEG_DIR)
    
    # 虹彩・瞳孔ラベル
    save_label(result, 'mask_iris', LABEL_OBB_DIR)
    save_label(result, 'mask_pupil', LABEL_OBB_DIR)
    
    # 5クラス教師データ
    save_label(result, 'iris_vis', LABEL_SEG_DIR)
    save_label(result, 'iris_occ', LABEL_SEG_DIR)
    save_label(result, 'pupil_vis', LABEL_SEG_DIR)
    save_label(result, 'pupil_occ', LABEL_SEG_DIR)

print("ラベル画像の保存完了")


ラベル画像を保存中...
保存中: 500/1992
保存中: 1000/1992
保存中: 1500/1992
ラベル画像の保存完了


## メタデータ保存


In [83]:
# データフレームを保存（画像・患者ID・パス情報のみ）
df_save = df[['image_id', 'filename', 'patient_id', 'original_size']].copy()
df_save.to_csv('image_metadata.csv', index=False)
print("image_metadata.csv に保存しました")

# 患者IDリストを保存
patient_list = df[['image_id', 'patient_id']].to_dict('records')
with open('patient_list.json', 'w') as f:
    json.dump(patient_list, f, indent=2)
print("patient_list.json に保存しました")

print("\nデータ前処理が完了しました！")
print(f"\n処理件数: {len(results)}")
print(f"患者数: {df['patient_id'].nunique()}")
print(f"fold_indices.json: 5-fold GroupKFold分割情報")
print(f"image_metadata.csv: 画像メタデータ")
print(f"patient_list.json: 患者IDリスト")


image_metadata.csv に保存しました
patient_list.json に保存しました

データ前処理が完了しました！

処理件数: 1992
患者数: 122
fold_indices.json: 5-fold GroupKFold分割情報
image_metadata.csv: 画像メタデータ
patient_list.json: 患者IDリスト
